### Loading Libraries

In [3]:
# Basic Libraries
import pandas as pd
import numpy as np
import pickle
import sys
from google.colab import files
import shutil
from google.colab import drive


### importing pre trained Hugging Face Transformer model
try:
  from sentence_transformers import SentenceTransformer
except:
  !pip install sentence_transformers
  from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
glove_model = SentenceTransformer('sentence-transformers/average_word_embeddings_glove.6B.300d')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a0aae0afed493b583e7d8293c057c0b977490266e501c00446f102f8941529a6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [7]:
def load_pickle_file(path):
  with open(path, 'rb') as fp:
    embedding_df = pickle.load(fp)
    fp.close()
  

    return embedding_df

In [48]:
drive.mount('/content/drive',force_remount = True)
print('### Loading Model pickle file ###')
pickle_path = '/content/drive/My Drive/quora_question_pairs_best_model.pkl'
loaded_pickle = load_pickle_file(pickle_path)
loaded_model = loaded_pickle['model']
features = loaded_pickle['features']

Mounted at /content/drive
### Loading Model pickle file ###


In [51]:
def clean_text(text):
  text = text.lower()
  text = re.sub("'", "", text) # to avoid removing contractions in english
  text = re.sub(r'\$\w*', '', text)
  text = re.sub(r'\'s', '', text)
  text = re.sub(r'<br />', '', text)
  text = re.sub(r'^RT[\s]+', '', text)
  #text = re.sub("@[A-Za-z0-9_]+","", text)
  #text = re.sub("#[A-Za-z0-9_]+","", text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub('[()!?]', ' ', text)
  text = re.sub('\[.*?\]',' ', text)
  text = re.sub("[^a-z0-9]"," ", text)
  text_links_removed = "".join([char for char in text if char not in string.punctuation])
  text_cleaned = " ".join([word for word in re.split('\W+', text_links_removed)])
  text = " ".join([lemmatizer.lemmatize(word) for word in re.split('\W+', text_cleaned)])
  text = text.strip()
  return text

def count_common_words(row):
    q1_words = set(map(lambda word: word.lower().strip(), row['question1'].split()))
    q2_words = set(map(lambda word: word.lower().strip(), row['question2'].split()))    
    return len(q1_words & q2_words)

def count_total_words(row):
    q1_words = set(map(lambda word: word.lower().strip(), row['question1'].split()))
    q2_words = set(map(lambda word: word.lower().strip(), row['question2'].split()))    
    return (len(q1_words) + len(q2_words))


def calculate_normalized_word_share(row):
    q1_words = set(map(lambda word: word.lower().strip(), row['question1'].split()))
    q2_words = set(map(lambda word: word.lower().strip(), row['question2'].split()))    
    return 1.0 * len(q1_words & q2_words) / (len(q1_words) + len(q2_words))

def is_not_present(a,b):
  a = a.lower()
  b = b.lower()
  if ('not' in a) | ('not' in b):
    if ('not' in a and 'not' not in b) | ('not' in b and 'not' not in a):
      return 1
    else:
      return -1
  else:
    return 0



def preprocess_data(final_df):
    from scipy.spatial.distance import cosine,cdist
    #final_df['question_1_embedding'] = final_df['question_1_embedding'].astype(object)
    #final_df['question_2_embedding'] = final_df['question_2_embedding'].astype(object)
    final_df['question_1_embedding'] = list(sbert_model.encode(final_df['question1']))
    final_df['question_2_embedding'] = list(sbert_model.encode(final_df['question2']))


    ## Length of each question
    final_df['q1_length'] = final_df['question1'].str.len()
    final_df['q2_length'] = final_df['question2'].str.len()

    ## Number of words in each question
    final_df['q1_word_count'] = final_df['question1'].apply(lambda x: len(x.split()))
    final_df['q2_word_count'] = final_df['question2'].apply(lambda x: len(x.split()))

    ## Number of common words in the dataset
    final_df['common_word_count'] = final_df.apply(count_common_words, axis=1)
    final_df['total_word_count'] = final_df.apply(count_total_words, axis=1)

    ## Normalized word share
    final_df['normalized_word_share'] = final_df.apply(calculate_normalized_word_share, axis=1)

    ## Euclidean Distance
    final_df['euclidean_dis'] = final_df.apply(lambda x: euclidean_dist(x.question_1_embedding, x.question_2_embedding), axis=1)

    ## Cosine Distance
    final_df['cosine_dis'] = final_df.apply(lambda x: cosine(x.question_1_embedding, x.question_2_embedding), axis=1)
    
    ## Negation
    final_df['is_negation'] = final_df.apply(lambda row: is_not_present(row['question1'],row['question2']),axis = 1)
    
    return final_df



def euclidean_dist(p1,p2):
  
  dist = np.linalg.norm(p1 - p2)

  return dist

def predict_similarity(q1,q2,features):

  predict_df = pd.DataFrame({'question1':q1,'question2':q2},index = [0])
  predict_df = preprocess_data(predict_df)
  predict_df = predict_df[features]
  pred_val = loaded_model.predict(predict_df)
  if pred_val == 1:
    print('The questions are similar')
  else:
    print('The questions are not similar')

In [56]:
q1 = 'What is the best/most memorable thing youve ever eaten and why?'
q2 = 'What is the most delicious dish youve ever eaten and why?'

predict_similarity(q1,q2,features)


The questions are similar


In [57]:
q1 = 'Im a 19-year-old. How can I improve my skills or what should I do to become an entrepreneur in the next few years?'
q2 = 'I am a 19 year old guy. How can I become a billionaire in the next 10 years?'

predict_similarity(q1,q2,features)

The questions are not similar
